In [2]:
import re
import string
from collections import defaultdict, Counter

import nltk as nltk
import pandas as pd
from nltk.corpus import wordnet
from nltk import pos_tag, WordNetLemmatizer, SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\20184025\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df_datascientist = pd.read_csv('../data/../data/DataScientist.csv', index_col=0)
df_datascientist.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


In [14]:
df_datascientist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3909 entries, 0 to 3908
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              3909 non-null   int64  
 1   Job Title          3909 non-null   object 
 2   Salary Estimate    3909 non-null   object 
 3   Job Description    3909 non-null   object 
 4   Rating             3909 non-null   float64
 5   Company Name       3909 non-null   object 
 6   Location           3909 non-null   object 
 7   Headquarters       3909 non-null   object 
 8   Size               3909 non-null   object 
 9   Founded            3909 non-null   int64  
 10  Type of ownership  3909 non-null   object 
 11  Industry           3909 non-null   object 
 12  Sector             3909 non-null   object 
 13  Revenue            3909 non-null   object 
 14  Competitors        3909 non-null   object 
 15  Easy Apply         3909 non-null   object 
 16  new tokens         3909 

In [4]:
df_datascientist['Job Description'][2]

"Decode_M\n\nhttps://www.decode-m.com/\n\nData Science Manager : Job Description\n\nWe’re hiring a Data Science Manager in our New York office to lead Decode_M’s Data Science team in developing new solutions for our clients and advancing the science of momentum through our proprietary product.\n\nThis position, based in New York City, requires an interest in managing Decode_M’s data science team, workflow and offerings. It demands excellent organizational skills, a natural curiosity, an eagerness to dive into the deep end, and a hunger to learn and grow while having a blast. Expect to learn a lot about cool companies, new products, and the latest in analytics, research and strategy.\n\nAbout You\nYou have an advanced degree in data science, mathematics, or another highly quantitative field, and are comfortable understanding and utilizing recent academic research in your work.\nYou have 3+ years of relevant work experience using applied statistics and/or machine learning. Bonus points i

In [5]:
def remove_urls(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    return re.sub(regex, '', text)

In [6]:
# remove urls
column_to_clean = 'Job Description'
df_datascientist[column_to_clean] = df_datascientist[column_to_clean].apply(remove_urls)
df_datascientist[column_to_clean]

0       ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...
1       At Noom, we use scientifically proven methods ...
2       Decode_M\n\n\n\nData Science Manager : Job Des...
3       Sapphire Digital seeks a dynamic and driven mi...
4       Director, Data Science - (200537)\nDescription...
                              ...                        
3904    About Us\n\nTachyon Technologies is a Digital ...
3905    Job description\nInterpret data, analyze resul...
3906    Job DescriptionThe Security Analytics Data Eng...
3907    The Security Analytics Data Engineer will inte...
3908    Help us transform patients' lives.\nAt UCB, we...
Name: Job Description, Length: 3909, dtype: object

In [7]:
df_datascientist[column_to_clean] = df_datascientist[column_to_clean].apply(lambda x: x.lower().strip())
df_datascientist[column_to_clean].head()

0    about hopper\n\nat hopper, we’re on a mission ...
1    at noom, we use scientifically proven methods ...
2    decode_m\n\n\n\ndata science manager : job des...
3    sapphire digital seeks a dynamic and driven mi...
4    director, data science - (200537)\ndescription...
Name: Job Description, dtype: object

In [8]:
stopwords = nltk.corpus.stopwords.words('english')
st = SnowballStemmer('english')
word_Lemmatized = WordNetLemmatizer()
tag_map = defaultdict(lambda: wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

def tokenize_and_remove(text):
    tokens = nltk.tokenize.word_tokenize(text)
    # remove punctuation and stopwords
    tokens = list(
        filter(lambda token: token not in string.punctuation and token not in stopwords and token.isalpha(),
               tokens))
    # stemming
    # tokens = [st.stem(token) for token in tokens]

    # lemmetize the words
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    tokens = [word_Lemmatized.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(tokens)]

    return tokens

tokenize_and_remove(df_datascientist['Job Description'][2])

['data',
 'science',
 'manager',
 'job',
 'description',
 'hire',
 'data',
 'science',
 'manager',
 'new',
 'york',
 'office',
 'lead',
 'data',
 'science',
 'team',
 'develop',
 'new',
 'solution',
 'client',
 'advance',
 'science',
 'momentum',
 'proprietary',
 'product',
 'position',
 'base',
 'new',
 'york',
 'city',
 'require',
 'interest',
 'manage',
 'data',
 'science',
 'team',
 'workflow',
 'offering',
 'demand',
 'excellent',
 'organizational',
 'skill',
 'natural',
 'curiosity',
 'eagerness',
 'dive',
 'deep',
 'end',
 'hunger',
 'learn',
 'grow',
 'blast',
 'expect',
 'learn',
 'lot',
 'cool',
 'company',
 'new',
 'product',
 'late',
 'analytics',
 'research',
 'strategy',
 'advance',
 'degree',
 'data',
 'science',
 'mathematics',
 'another',
 'highly',
 'quantitative',
 'field',
 'comfortable',
 'understanding',
 'utilizing',
 'recent',
 'academic',
 'research',
 'work',
 'year',
 'relevant',
 'work',
 'experience',
 'use',
 'applied',
 'statistic',
 'machine',
 'learn',


In [9]:
df_datascientist['new tokens'] = df_datascientist[column_to_clean].apply(tokenize_and_remove)
df_datascientist.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,new tokens
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"about hopper\n\nat hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,"[hopper, hopper, mission, make, book, travel, ..."
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"at noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,"[noom, use, scientifically, proven, method, he..."
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),decode_m\n\n\n\ndata science manager : job des...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True,"[data, science, manager, job, description, hir..."
3,3,Data Analyst,$111K-$181K (Glassdoor est.),sapphire digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,"[sapphire, digital, seek, dynamic, drive, data..."
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"director, data science - (200537)\ndescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,"[director, data, science, description, edelman..."


In [10]:
all_tokens = []
for _, value in df_datascientist['new tokens'].iteritems():
    all_tokens.extend(value)

count: Counter = Counter(all_tokens)

In [11]:
count.most_common(100)

[('data', 35645),
 ('experience', 21462),
 ('work', 14942),
 ('team', 10144),
 ('business', 10101),
 ('skill', 7072),
 ('year', 7013),
 ('use', 6676),
 ('include', 6473),
 ('science', 6182),
 ('development', 6003),
 ('model', 5851),
 ('analysis', 5785),
 ('develop', 5763),
 ('ability', 5628),
 ('require', 5231),
 ('technology', 5161),
 ('solution', 4973),
 ('product', 4971),
 ('design', 4965),
 ('support', 4956),
 ('system', 4860),
 ('project', 4840),
 ('knowledge', 4795),
 ('provide', 4711),
 ('process', 4670),
 ('job', 4666),
 ('analytics', 4646),
 ('learn', 4638),
 ('research', 4525),
 ('opportunity', 4322),
 ('service', 4263),
 ('strong', 4253),
 ('information', 4235),
 ('company', 4234),
 ('new', 4197),
 ('technical', 4040),
 ('management', 4030),
 ('requirement', 3997),
 ('scientist', 3835),
 ('build', 3825),
 ('tool', 3800),
 ('report', 3673),
 ('need', 3633),
 ('environment', 3560),
 ('application', 3463),
 ('engineering', 3431),
 ('customer', 3417),
 ('position', 3393),
 ('mac

In [15]:
count['dedicated']

34

In [17]:
count['modeling']

578

In [16]:
count['python']

2636

In [5]:
# import string
# list(filter(lambda x: x not in string.punctuation, text))

In [ ]:


def clean_jobdescriptions(df, column_to_clean: str):
    # 3. Remove Tweets with URLs
    def Find_url(string):
        # URL is defined as https:// with at least 2 chars behind this
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        url = re.findall(regex, string)
        return len(url) > 0

    def Find_url_word_small(string):
        # URL is defined as https:// with at least 2 chars behind this
        regex = r"url"
        url = re.findall(regex, string)
        return len(url) > 0

    def Find_url_word(string):
        # URL is defined as https:// with at least 2 chars behind this
        regex = r"URL"
        url = re.findall(regex, string)
        return len(url) > 0

    df["URL"] = False
    df["URL"] = df[column_to_clean].apply(lambda x: Find_url(x) or Find_url_word(x) or Find_url_word_small(x))
    df = df[df["URL"] != True]
    print(f"Non-URL proportion {round(len(df) * 100 / len(df), 4)} % with a total of {len(df)}")

    # 3.1 Change all the text to lower case
    df[column_to_clean] = [entry.lower() for entry in df[column_to_clean]]

    # 4. Split Tweets into words
    df[column_to_clean] = df[column_to_clean].apply(lambda x: x.split(' '))

    # 6. Remove stop words and word-stemming
    tag_map = defaultdict(lambda: wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    for index, entry in zip(df.index.tolist(), df[column_to_clean]):
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
                Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
        df.loc[index, 'text_final'] = str(Final_words)

    # 5. Select Tweets with least 2 words
    # TODO: Check if step necessary because it was commented out
    df["Words"] = False
    df["Words"] = df['text_final'].apply(lambda x: True if len(x) > 1 else False)  # Threshold for length tweets
    df = df[df["Words"] == True]
    print(f"Long proportion {round(len(df) * 100 / len(df), 4)} % with a total of {len(df)}")